This notebook is for scraping Denton's 'Enough Rope' transcripts.

In [1]:
import csv
import os
import time

from lxml import html
import requests

OUTPUT_DIR = 'data/enough-rope-transcripts'

In [2]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
page = requests.get('http://www.abc.net.au/tv/enoughrope/pastepisodes/name.htm')
tree = html.fromstring(page.content)

PermissionError: [Errno 13] Permission denied: '/data'

In [10]:
for linkEl in tree.xpath('//div[@class="txtCommon"]//a'):
    link = linkEl.attrib.get('href', None)
    if link:
        page = requests.get(link)
        page.encoding = 'windows-1252'
        content = html.fromstring(page.text)
        title = content.xpath('//div[@id="storyFeature"]//h2//text()')[0]
        date = content.xpath('//span[@class="date"]//text()')[0]
        print(title, date)
        speakers = content.xpath('//div[@id="transContent"]//p//span[@class="talent"]')
        file_path = os.path.join(OUTPUT_DIR, '{}-{}.csv'.format(title, date))
        if os.path.exists(file_path):
            continue
        with open(file_path, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Speaker', 'Text'])
            row = []
            for speaker in speakers:
                if speaker is None:
                    continue
                speaker_name = speaker.text.strip()
                if speaker_name.endswith(':'):
                    speaker_name = speaker_name[:-1]
                text = speaker.xpath('./following-sibling::text()')[0].strip()
                writer.writerow([speaker_name, text])

        time.sleep(2)  # Don't spam the server or we might get blocked (plus it's not  nice)

Chris Abani  - 6 October 2008


FileNotFoundError: [Errno 2] No such file or directory: '/data/enough-rope-transcripts/Chris Abani-\xa0-\xa06 October 2008.csv'